In [1]:
# importing the necessary packages
import numpy as np
import pandas as pd
import os
import spacy
import pickle

# if necessary download the pipeline
#!python -m spacy download nl_core_news_sm

import nl_core_news_sm
nlp = nl_core_news_sm.load()


# setting the main file path to where all files are
os.chdir("C:\\Users\\SJNic\\Documents\\University\\Master\\Thesis\\Thesis Code")

#texts = pd.read_csv("NewProcessingData/Document_Text.csv", index_col = 0)
texts = pd.read_pickle("Texts/newDocument_Text.pkl")#, index_col = 0)
texts

,text
key,
0000a2fd-0427-4949-a3b4-e39f4635f421,"cx,t,e_0079t, fric,cd - -1.01 £cLLiot f‘tc,Apt..."
0005b23b-f2be-4b55-9f83-dcf966cb3b29,
00068e84-19ef-4db9-8d2a-7e51ff69e18c,A 201 (1-5--T- Utrecht Amendement - Een vergad...
00096dc9-44fd-4ba2-aa89-6ece5f79a44b,Motie: Beargumenteren komt voor bezuinigen De ...
00097ca8-b918-4dd3-99bf-9728d1fd5ee4,Bijlage 1 BI-zone 4 Merwedekanaalzone Zuid 2011
...,...
fff65d87-1368-42a0-85a0-028a36b66820,StadsOntwikkeling Bijlage bestemmingsplan Best...
fff9faf4-7f86-43af-a065-d803b85e1268,Verordening U-pas 2019 gemeente Utrecht De raa...
fffe5196-feb7-4be1-8ef7-cc55a79e16ac,


# Lemmatize the text

In [4]:
from tqdm import tqdm  # for a progress bar

# set max length higher in order to process all text
# 1m ~= 1 GB
nlp.max_length = 8000000

lemmas = []

# loop through all the texts and convert them to seperate lemmas using a spacy
# also with a nice progress bar (took 4 hours for me to process)
for i in tqdm(range(len(texts["text"].values))):
        text = str(texts["text"].values[i])
        processed_text = nlp(text.lower())
        lemmatized_text = [token.lemma_ for token in processed_text if token.pos_ == 'NOUN' or token.pos_ == 'VERB' or token.pos_ == 'ADJ' or token.pos_ == 'PROPN']
        lemmas.append(lemmatized_text)
        
# store the lemmas in the dataframe and save it to a csv  
texts["lemmas"] = lemmas
#texts[["key", "lemmas"]].to_csv("similarities/lemmas.csv")
texts

100%|██████████| 44613/44613 [6:25:51<00:00,  1.93it/s]    


,text,lemmas
key,,
0000a2fd-0427-4949-a3b4-e39f4635f421,"cx,t,e_0079t, fric,cd - -1.01 £cLLiot f‘tc,Apt...","[cx, e_0079, t, fric, cd, -1.01, £, clliot, f,..."
0005b23b-f2be-4b55-9f83-dcf966cb3b29,,[]
00068e84-19ef-4db9-8d2a-7e51ff69e18c,A 201 (1-5--T- Utrecht Amendement - Een vergad...,"[utrecht, amendement, vergadering, zijn, gemee..."
00096dc9-44fd-4ba2-aa89-6ece5f79a44b,Motie: Beargumenteren komt voor bezuinigen De ...,"[motie, beargumenter, bezuinig, gemeenteraad, ..."
00097ca8-b918-4dd3-99bf-9728d1fd5ee4,Bijlage 1 BI-zone 4 Merwedekanaalzone Zuid 2011,"[bijlage, bi-zone, merwedekanaalzon, zuid]"
...,...,...
fff65d87-1368-42a0-85a0-028a36b66820,StadsOntwikkeling Bijlage bestemmingsplan Best...,"[stadsontwikkeling, bijlage, bestemmingsplan, ..."
fff9faf4-7f86-43af-a065-d803b85e1268,Verordening U-pas 2019 gemeente Utrecht De raa...,"[verordening, u-pas, gemeente, utrecht, raad, ..."
fffe5196-feb7-4be1-8ef7-cc55a79e16ac,,[]


In [ ]:
# first we have to do some further cleaning of the lemmas
def cleanText(texts):
    newTexts = []
    for text in texts:

        text = re.sub("[^a-zA-Z]", "", text)
        if " " in text or "" == text:
            continue
        newTexts.append(text)
    return newTexts

In [ ]:
lemmas["lemmas"] = lemmas["lemmas"].apply(cleanText)

In [5]:
# save the lemmas
texts["lemmas"].to_pickle("similarities/newLemmas.pkl")

# Converting to item_lemmas
This is not necessary anymore, but if someone wants to use it then it is possible

In [11]:
# read the lemmas again
from tqdm import tqdm
lemmas = pd.read_pickle("similarities/newLemmas.pkl").to_frame()

# open the itemsInformation
with open("NewProcessingData/itemsInformation.txt", "rb") as fp:   
    itemsInformation = pickle.load(fp)
with open("NewProcessingData/documentsInformation.txt", "rb") as fp:   
    documentsInformation = pickle.load(fp)

In [12]:
lemmas

,lemmas
key,
0000a2fd-0427-4949-a3b4-e39f4635f421,"[cx, e_0079, t, fric, cd, -1.01, £, clliot, f,..."
0005b23b-f2be-4b55-9f83-dcf966cb3b29,[]
00068e84-19ef-4db9-8d2a-7e51ff69e18c,"[utrecht, amendement, vergadering, zijn, gemee..."
00096dc9-44fd-4ba2-aa89-6ece5f79a44b,"[motie, beargumenter, bezuinig, gemeenteraad, ..."
00097ca8-b918-4dd3-99bf-9728d1fd5ee4,"[bijlage, bi-zone, merwedekanaalzon, zuid]"
...,...
fff65d87-1368-42a0-85a0-028a36b66820,"[stadsontwikkeling, bijlage, bestemmingsplan, ..."
fff9faf4-7f86-43af-a065-d803b85e1268,"[verordening, u-pas, gemeente, utrecht, raad, ..."
fffe5196-feb7-4be1-8ef7-cc55a79e16ac,[]


In [18]:
from tqdm import tqdm
item_lemmas = pd.DataFrame()
item_lemmas["itemID"] = itemsInformation.keys()
item_lemmas = item_lemmas.set_index("itemID")
item_lemmas["lemmas"] = np.empty((len(item_lemmas), 0)).tolist()

# loop through every item
for key in tqdm(itemsInformation):
    
    docs = itemsInformation[key]["Documents"]
    
    # if there is at least one document then add its lemmas to the item's lemmas 
    if len(docs) >= 1:
        
        # loop through every document
        for doc in docs:
            # first get the unique version of this document
            uniqueDocID = documentsInformation[doc["id"]]["unique id"]
            #print(uniqueDocID)
            
            # sometimes the value is dict so retrieve the ID from that referenece        
            if uniqueDocID is dict or str(type(uniqueDocID)) == "<class 'dict'>":
                uniqueDocID = uniqueDocID["id"]
            
            # some documents have not been fully processed so skip those
            try:
                item_lemmas.loc[key]["lemmas"] = item_lemmas.loc[key]["lemmas"] + lemmas.loc[uniqueDocID]["lemmas"]
            except:
                continue
        #break
    else :
        continue
    
item_lemmas

100%|██████████| 15314/15314 [00:10<00:00, 1428.50it/s]


,lemmas
itemID,
f0607435-a601-4a06-85d4-63dd17ca3ec6,[]
c6b49408-1158-495b-b69e-6137da9da1e7,[]
0d47334d-7b3a-4b62-9ab6-7ff63fe49ebf,[]
f804c38b-59f3-40c1-a9f3-c28c8b6078cc,[]
400094ee-0e1f-432f-b1e4-ba41abdd516b,[]
...,...
8c20e0fa-9227-4e77-90f5-1de4b96696f1,"[actieplan, zelfbeheer, groen, februari, utrec..."
a6a6d28b-0715-419a-93ba-b95ce6ab6d50,[]
9e63ca83-e8a7-48be-abfa-227b0f9e6116,"[voorstel, gemeenteraad, utrechts, sportagenda..."


In [19]:
# save the item lemmas
item_lemmas.to_pickle("Texts/newItem_lemmas.pkl")